# Content based pipe

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
import joblib
import sys
import jdc

%load_ext autoreload
%autoreload 2 

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-fd1313zo because the default path (/home/pjavier/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
from resype.preprocessing import *

In [5]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [6]:
transaction_list, user_df = create_user_feature() # for example only
item_df = create_item_feature(num_features = 300)

In [7]:
# transaction list
transaction_list.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
# user ids and features
user_df.head()

,userId,u_1,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,...,u_11,u_12,u_13,u_14,u_15,u_16,u_17,u_18,u_19,u_20
0,1,85,29,42,83,47,26,90,45,55,...,17,68,22,7,40,22,1,0,0,0
1,2,3,0,0,7,0,1,11,10,10,...,1,17,1,1,4,0,0,4,3,0
2,3,11,4,5,9,4,5,14,2,7,...,8,16,5,0,15,1,0,0,0,0
3,4,29,6,10,104,19,58,25,27,38,...,4,120,7,10,12,16,4,1,2,0
4,5,8,6,9,15,7,11,9,12,9,...,1,25,3,2,2,5,0,3,0,0


In [9]:
# item ids and features
item_df.head()

,movieId,i_1,i_2,i_3,i_4,i_5,i_6,i_7,i_8,i_9,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.513025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load ContentBasedModel

In [45]:
from resype.content_based import ContentBasedModel

In [46]:
cb = ContentBasedModel(user_df,
                        item_df,
                        transaction_list,
                        item_id_name='movieId',
                        user_id_name='userId',
                        target_name='rating',
                        timestamp_name='timestamp')

In [47]:
cb.df # user-movie

,userId,movieId,rating,timestamp,u_1,u_2,u_3,u_4,u_5,u_6,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
0,1,1,4.0,964982703,85,29,42,83,47,26,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,1,3,4.0,964981247,85,29,42,83,47,26,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,1,6,4.0,964982224,85,29,42,83,47,26,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,1,47,5.0,964983815,85,29,42,83,47,26,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,1,50,5.0,964982931,85,29,42,83,47,26,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,267,66,56,411,151,119,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
100832,610,168248,5.0,1493850091,267,66,56,411,151,119,...,0.0,0.0,0.000000,0.0,0.0,0.245020,0.0,0.0,0.0,0.000000
100833,610,168250,5.0,1494273047,267,66,56,411,151,119,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
100834,610,168252,5.0,1493846352,267,66,56,411,151,119,...,0.0,0.0,0.000000,0.0,0.0,0.291784,0.0,0.0,0.0,0.282198


## Train-test Split

In [48]:
#Do your train and test split
cb.split_train_test(train_ratio = 0.7) #To split the data

In [49]:
cb.df_test

,userId,movieId,rating,timestamp,u_1,u_2,u_3,u_4,u_5,u_6,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
0,1,1,4.0,964982703,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,1,3,4.0,964981247,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
7,1,110,4.0,964982176,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
10,1,163,5.0,964983650,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
12,1,223,3.0,964980985,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100825,610,161634,4.0,1493848362,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
100828,610,163981,3.5,1493850155,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
100830,610,166528,4.0,1493879365,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
100831,610,166534,4.0,1493848402,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [50]:
cb.df_train

,userId,movieId,rating,timestamp,u_1,u_2,u_3,u_4,u_5,u_6,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
60,1,1073,5.0,964981680,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.707217,0.000000,0.0,0.0,0.0
49,1,1009,3.0,964981775,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
105,1,1625,5.0,964983504,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
215,1,3450,5.0,964981144,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
155,1,2414,3.0,964982513,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100800,610,147662,3.0,1479544214,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.283971,0.415069,0.0,0.0,0.0
100641,610,104241,3.0,1479544709,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
99951,610,6283,4.5,1493845616,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100040,610,8142,5.0,1479545324,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


## Fit model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=202109, n_jobs=-1)
cb.fit_ml_cb(model)

In [ ]:
# show trained model
cb.model

## Predict on Test Set

In [41]:
cb.df_test

,userId,movieId,rating,timestamp,u_1,u_2,u_3,u_4,u_5,u_6,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
4,1,50,5.0,964982931,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
12,1,223,3.0,964980985,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15,1,260,5.0,964981680,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
17,1,316,3.0,964982310,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
18,1,333,5.0,964981179,85,29,42,83,47,26,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100822,610,160571,3.0,1493848537,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100823,610,160836,3.0,1493844794,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.287034,0.0,0.0,0.0,0.0
100825,610,161634,4.0,1493848362,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100828,610,163981,3.5,1493850155,267,66,56,411,151,119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [43]:
#Predict on the test data
preds_array = cb.reco_ml_cb_tt() #To make predictions as an array
print(len(preds_array))
preds_array

30256


array([4.        , 4.34      , 4.805     , ..., 3.75722222, 4.14726984,
       3.805     ])

## Get Recommendations

In [ ]:
cb.

In [ ]:
user_ = user_df.drop(['userId'], axis=1)

In [ ]:
user_ = user_.sample(10)

In [ ]:
item_ = item_df.drop(['movieId'], axis=1)

In [ ]:
reco_mtx = reco_ml_cb(user_, item_, item_ids, model_fit)
reco_mtx

In [ ]:
reco_inds = np.array(reco_mtx.index)
reco_inds

In [ ]:
um = _.drop('timestamp', axis=1)
um.columns = ['user_id', 'item_id', 'rating']

rs = Resype(um)
original_um = rs.construct_utility_matrix()
original_um = original_um.loc[reco_inds[:], :]
original_um = original_um[reco_mtx.columns]

In [ ]:
reco_mtx.shape, original_um.shape

In [ ]:
original_um = original_um.fillna(0)
original_um

In [ ]:
reco_mtx = reco_mtx.fillna(0)
reco_mtx

## Get recommendations

In [ ]:

get_rec(reco_mtx, original_um, user_list = [0, 2], top_n = 3)

## Evaluate

In [ ]:
evaluate_arrays(preds_array, df, indx_test) #MSE and MAE

## Cross Validation

In [ ]:
df

In [ ]:
mse, mae = cross_val(df, 5, rs_model1, split_method='random')

In [ ]:
mse, mae